In [171]:
from sndaq.reader import SN_PayloadReader
from sndaq.datetime_ns import datetime_ns as dt_ns

from sndaq.buffer import windowbuffer
from sndaq.detector import Detector
import numpy as np
import sys

import json
import urllib
import timeit

In [313]:
i3 = Detector('./data/full_dom_table.txt')
filename = './data/sndata-spts-209058_001-010/sn_209058_000001_239420_529095.dat'
limit = 10

base_buffer = windowbuffer(20)
step = int(2e7)
depth = 2000
data_col = np.zeros((5160, depth), dtype=np.uint16)
payloads_read = np.zeros(5160, dtype=np.uint16)

start_time = timeit.default_timer()

with SN_PayloadReader(filename) as rdr:
    n_read = 1
    pay = next(rdr)
    file_start_time = pay.utime
    base_bins = np.arange(file_start_time, 
                          file_start_time+step*depth,
                          step)
    
    while not isvalid_dom(pay.dom_id):
        pay = next(rdr)
   
    row = process_scalers(pay, base_bins)
    data_col[i3.get_dom_idx(pay.dom_id)] += row
    payloads_read[i3.get_dom_idx(pay.dom_id)] += 1
    
    for pay in rdr:
        if isvalid_dom(pay.dom_id):
            row = process_scalers(pay, base_bins)
            data_col[i3.get_dom_idx(pay.dom_id)] += row
            payloads_read[i3.get_dom_idx(pay.dom_id)] += 1
        
        if pay.utime > base_bins[0]:
            base_buffer.append(data_col[:,0])
            base_bins = np.roll(base_bins, -1)
            base_bins[-1] = base_bins[-2] + step
            data_col = np.roll(data_col, -1, axis=1)
            data_col[:, -1] = 0
                                  
        if np.all(payloads_read>=n_read) or np.any(payloads_read>=2):
            break
            
elapsed = timeit.default_timer() - start_time
processed = dt_ns.from_utime(pay.utime) - dt_ns.from_utime(file_start_time) 

In [314]:
print('Elapsed time: {0}'.format(elapsed))
print('Processed time: {0}'.format(processed.to_ns/1e9))

Elapsed time: 22.50619279104285
Processed time: 1.03708889


In [319]:
%%timeit
with SN_PayloadReader(filename) as rdr:
    for i, pay in enumerate(rdr):
        if i >= 5129:
            break

33.2 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


31.21212121212121

In [64]:
def isvalid_dom(dom_id):
    return np.any(i3._dom_id_sorted == dom_id)

In [109]:
def process_scalers(payload, time_bins):
    """time_bins is base (2ms) time bins
    Could be changed to increment bin time as np.uint16 rather than thru array elements
    """
    base_counts = np.zeros(time_bins.size, dtype=np.uint16)
    
    itr_time = iter(time_bins)  
    start_utime = next(itr_time)  # Leading edge of first bin
    current_bin = (start_utime, next(itr_time))
    
    idx_base = 0
    frac = 1
    
    clock_cycle = 250 * 2**16
    scaler_bin = (payload.utime, payload.utime + clock_cycle)
    while scaler_bin[0] > current_bin[1]:  # Should only affect first scalars in file
        current_bin = (current_bin[1], next(itr_time))
        idx_base += 1
    
    for i, scaler in enumerate(payload.scaler_bytes):
        scaler_bin = (payload.utime + i*clock_cycle, payload.utime + (i+1)*clock_cycle)

        if scaler_bin[1] <= current_bin[1]:
            base_counts[idx_base] += scaler

        elif scaler_bin[1] > current_bin[1] and scaler_bin[0] < current_bin[1]:
            frac = (current_bin[1] - scaler_bin[0])/clock_cycle
            base_counts[idx_base] += np.uint16(0.5+frac*scaler)  # Ensures proper rounding, see numpy rint/around
            current_bin = (current_bin[1], next(itr_time))
            idx_base += 1
            base_counts[idx_base] += np.int16(0.5+(1-frac)*scaler)
    
    return base_counts

In [283]:
# dt_s=16384000
# test = np.array(np.arange(dt_s, 200*dt_s, dt_s, dtype=int))
# vec_scaler2base(test)


(array([518, 519, 520, 521, 521, 522, 523, 524, 525, 525, 526, 527, 528,
        529, 530, 530, 531, 532, 533, 534, 534, 535, 536, 537, 538, 539,
        539, 540, 541, 542, 543, 543, 544, 545, 546, 547, 548, 548, 549,
        550, 551, 552, 552, 553, 554, 555, 556, 557, 557, 558, 559, 560,
        561, 561, 562, 563, 564, 565, 566, 566, 567, 568, 569, 570, 570,
        571, 572, 573, 574, 575, 575, 576, 577, 578, 579, 579, 580, 581,
        582, 583, 584, 584, 585, 586, 587, 588, 588, 589, 590, 591, 592,
        593, 593, 594, 595, 596, 597, 598, 598, 599, 600, 601, 602, 602,
        603, 604, 605, 606, 607, 607, 608, 609, 610, 611, 611, 612, 613,
        614, 615, 616, 616, 617, 618, 619, 620, 620, 621, 622, 623, 624,
        625, 625, 626, 627, 628, 629, 629, 630, 631, 632, 633, 634, 634,
        635, 636, 637, 638, 638, 639, 640, 641, 642, 643, 643, 644, 645,
        646, 647, 647, 648, 649, 650, 651, 652, 652, 653, 654, 655, 656,
        656, 657, 658, 659, 660, 661, 661, 662, 663

### Begin Scratch

In [3]:
filename = './data/sndata-spts-209058_001-010/sn_209058_000001_239420_529095.dat'

with SN_PayloadReader(filename) as rdr:
    previous = next(rdr)
    dom_id = previous.dom_id
    print(dt_ns.from_utime(previous.utime))
    print(len(previous.data_bytes))
    
    for i, pay in enumerate(rdr):
        if pay.dom_id == dom_id:
            break
        
print(dt_ns.from_utime(pay.utime))
print()


2021-07-01 21:08:05.1073427000
670
2021-07-01 21:08:06.1755819640



In [8]:
# Future Unit tests

# Assert Equal (casting to int may be needed), assumes pay and previous have the same dom ID
(pay.domclock - previous.domclock)/65536
previous.scaler_length


delta_t = dt_ns.from_utime(pay.utime) - dt_ns.from_utime(previous.utime)
#assert equal to 7 sig figs
print(int(delta_t.to_ns / previous.scaler_length)*1e-9)
25e-9 * 65536  #0.0016384 == 1.6384ms

# Assert True/Equals
np.where(i3.dom_table['mbid']==0x88ce18beda45)[0][0] == i3.get_dom_idx(0x88ce18beda45)

# Check to make sure first payload that begins 2ms after collection start has no scalers in first 2ms bin

0.001638403


0.0016384

In [69]:
N = 6
dom_ids = []
sizes = []
with SN_PayloadReader(filename) as rdr:
    pay = next(rdr)
    dom_ids.append(hex(pay.dom_id))
    sizes.append(pay.scaler_length)
    for i, pay in enumerate(rdr):
        if i >= N-1:
            break
        if pay.dom_id in dom_ids:
            print('FOUND DUPLICATE')
            break
        else:
            dom_ids.append(hex(pay.dom_id))
            sizes.append(pay.scaler_length)

for dom_id, s in zip(dom_ids, sizes):
    print(f'{dom_id} \t {s}')
        

0x88ce18beda45 	 652
0xfb388b597810 	 664
0x57a6cb44117e 	 640
0x4bb001dfe1e2 	 640
0xc573bd2aad72 	 656
0x17e952cb36a9 	 672


#### Pseudo-code for payload handling

- Obtain list of files
- Obtain first file (idx=0), select
- Process selected file
  NOTE: payloads written to file are sorted monotonically in time.  
  - Read first payload, record start time
  - Construct timestamps using start time and 2ms known binning, create equally sized deep buffer
  - Interpolate scalars to 2ms using sndaq method
  - perform row lookup via dom_id
  - Add scalars to looked-up row of deep buffer.
  - Read next payload, repeat interpolate scalars until all doms have issued 1 payload. (For now, assume that all doms issue payload 1 before any dom issues payload 2)
  - Read columns of 2ms hits into analysis buffer
- Mark file as processed
- Search for new files
- If run continues, obtain next file (idx+=1) or wait
- If run stops, shutdown.

In [28]:
file_start = 0
n_rec = 3
payload_start = 0
data_col = np.zeros(1000, 5160, dtype=np.uint16)
base_buffer = windowbuffer(20)

with SN_PayloadReader(filename) as rdr:    
    for i, pay in zip(range(n_rec), rdr):
        if file_start == 0:
            file_start = pay.utime
            
        if payload_start == 0:
            payload_start = pay.utime
        
        
        for scaler in pay.scaler_bytes:
            print(scaler)
        
        print(pay)

        

0
0
0
0
0
0
0
1
0
0
0
2
0
1
0
0
0
0
1
0
2
1
0
0
2
1
1
0
0
1
1
0
0
0
0
0
1
0
1
0
1
1
1
1
0
0
0
2
1
0
0
1
0
1
0
1
1
1
0
0
1
0
0
0
1
0
0
0
1
0
2
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
1
0
3
0
2
1
1
1
0
0
0
0
1
0
1
0
0
0
0
0
0
0
0
1
0
0
2
0
0
0
0
1
1
1
0
1
1
0
0
1
0
0
0
0
1
1
1
3
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
2
0
0
1
0
1
0
0
0
0
0
1
1
1
0
0
0
0
0
2
1
0
1
0
1
1
1
0
0
0
0
0
2
0
0
1
0
1
1
1
0
0
0
1
0
0
0
1
0
0
2
0
4
1
0
1
2
1
0
0
1
1
1
2
1
0
0
1
1
3
1
2
1
1
0
1
0
0
0
0
0
0
0
0
0
0
0
0
1
2
1
0
0
1
2
1
0
0
0
0
0
0
2
1
0
2
1
0
2
0
0
0
0
0
1
0
0
0
1
0
1
1
0
0
0
1
1
0
0
1
2
1
0
1
0
1
1
0
1
0
1
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
2
0
1
0
0
0
1
0
1
0
1
0
0
0
0
0
0
0
1
0
2
0
0
0
1
0
1
2
0
0
0
0
1
0
1
0
1
1
0
1
0
0
0
0
1
4
1
2
1
0
0
0
0
2
2
0
0
1
0
1
0
0
0
0
0
0
0
0
0
0
2
0
3
0
0
0
2
0
1
1
0
0
0
2
0
1
0
2
1
2
0
0
1
0
2
0
1
2
0
0
1
1
1
0
3
1
0
0
0
1
0
0
0
0
0
1
0
0
2
1
1
0
1
1
0
1
2
0
1
1
0
0
0
0
0
1
1
0
0
1
0
0
0
1
3
1
0
0
0
1
1
0
0
0
0
0
1
1
1
2
2
2
0
1
0
0
0
0
2
2


In [14]:
domclk_cycles_to_readout = 2**16
ns_per_domclk_cycle = 25
dt_per_scaler  = domclk_cycles_to_readout * ns_per_domclk_cycle
dt_per_bin = int(2e-3 * 1e9)
print((dt_per_scaler, dt_per_bin))

(1638400, 2000000)


In [12]:
reqest_excluded_doms = False
if reqest_excluded_doms:
    run_no = 135570
    url = 'https://live.icecube.wisc.edu/run_doms/{0}/'.format(run_no)
    # Edit these credentials before running
    params = {
        'user': 'icecube',
        'pass': 'skua',
        'run_number': run_no
    }

    if params['user'] == 'REDACTED' or params['pass'] == 'REDACTED':
        raise RuntimeError('Invalid User and/or pass for i3Live POST request')

    data = urllib.parse.urlencode(params).encode("utf-8")
    req = urllib.request.Request(url)
    with urllib.request.urlopen(req,data=data) as f:
        response = f.read()



    # Parse JSON into a python dict and print some of it
    d = json.loads(response)
    excluded_doms = []
    for coord, dom_dict in d['unconfigured_doms'].items():
        mbid = int(dom_dict['mb_id'], 16)
        if mbid not in excluded_doms:
            excluded_doms.append(mbid)         

    for alert, dom_dict in d['dropped_doms'].items():
        mbid = int(dom_dict[0]['dom_mbid'], 16)
        if mbid not in excluded_doms:
            excluded_doms.append(mbid)

    excludedDoms = np.asarray(excluded_doms, dtype=('u8'))
    print(excludedDoms.size)

array([ 896, 4553,  369, ..., 4251, 3490, 2689])

In [130]:
# Vectorized scaler processing
# Turned out that non-vectorized version was slightly faster.
dt_b=int(2e7)
clock_cycle = 16384000
base_time=np.arange(file_start_time, file_start_time+2000*dt_b, dt_b, dtype=int)

def scaler2base(scaler, scaler_time, scaler_dt=clock_cycle, time=base_time, dt=dt_b):
    idx = np.searchsorted(time, scaler_time, side="left") -1
    if time[idx]+dt >= scaler_time + scaler_dt:
        return idx, scaler, 0
    elif scaler_time + scaler_dt > time[idx] + dt and scaler_time < time[idx]+dt: 
        frac = (time[idx]+dt - scaler_time)/scaler_dt
        return idx, np.int16(0.5+frac*scaler), np.int16(0.5+(1-frac)*scaler)
    elif scaler_time > time[idx]+dt:
        return idx, -1, -1
vec_scaler2base = np.vectorize(scaler2base, excluded=['scaler_dt', 'time', 'dt'])

scalers = np.frombuffer(pay.scaler_bytes, dtype=np.uint8)
scaler_bins = np.arange(pay.utime, pay.utime+pay.scaler_length*clock_cycle, clock_cycle, dtype=int)
vec_scaler2base(scalers, scaler_bins)

96.5 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [113]:
np.searchsorted(doms['mbid'], pay.dom_id, sorter=I)

941

In [114]:
%%timeit
idx = np.where(doms['mbid']==pay.dom_id)[0][0]
# idx = (doms['mbid']==pay.dom_id).nonzero()[0]

9.2 µs ± 83.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
empty(dom_map['mbid'][dom_map['mbid']==1])

NameError: name 'empty' is not defined

In [129]:
np.flatnonzero(doms['mbid']==1)[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [111]:
pay.dom_id

49220100833824

In [16]:
doms.size

5160